<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/energyopt/PyPSAEarthOffshoreField.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyPSA-Earth Example: Powering an Offshore Field
This notebook demonstrates how to use PyPSA-Earth to supply a new 100 MW offshore field from the Norwegian electricity grid.

# Background
Norway has announced a strategy to electrify offshore oil and gas installations using power delivered from the mainland grid.
Supplying an offshore field from land can significantly reduce emissions because gas turbines or diesel generators are replaced by electricity generated onshore, typically from low-carbon sources.
This example illustrates how a new 100 MW field could be connected to the existing grid.

# Modeling approach
PyPSA-Earth builds detailed electricity networks from open data. Here we restrict the network to Norway and then add an additional bus representing the offshore facility.
The connection to land is modeled as a `Link` with a nominal capacity of 100 MW. A constant 100 MW `Load` is attached to the offshore bus to represent the field's power requirement.
Running `lopf` performs a linear optimal power flow to verify that the network can supply the new demand.

# Theory
Electrifying offshore fields allows operators to replace onboard generation from gas turbines with mainland electricity, reducing greenhouse gas emissions and improving efficiency.
In network studies PyPSA-Earth represents onshore and offshore locations as buses connected via high-voltage links. The optimal power flow determines whether the grid can supply the new demand without violating transmission constraints.

In [ ]:
!pip install pypsa pypsa-earth

In [ ]:
import pypsa
import pypsa_earth as earth

# Build a PyPSA-Earth network for Norway
n = earth.get_electricity_network(continent='Europe', country_codes=['NO'])

# Add bus representing the offshore field (example coordinates)
n.add('Bus', 'offshore_field', x=2.5, y=62.0, country='NO', carrier='AC')

# Connect the field to an onshore bus
onshore_bus = n.buses.index[0]
n.add('Link', 'grid_to_field', bus0=onshore_bus, bus1='offshore_field', p_nom=100, efficiency=0.95)

# Add constant demand of 100 MW at the offshore field
n.add('Load', 'field_load', bus='offshore_field', p_set=100)

# Run the linear optimal power flow
n.lopf(n.snapshots)

In [ ]:
n.links_t.p0.head()

In [ ]:
n.loads

# Interpreting the results
In this simple demonstration the load is always 100 MW and is served via the onshore grid with an assumed efficiency of 95%.
Real projects would model variable demand, cable losses, and potential constraints in the mainland grid.
Nevertheless, this approach shows how PyPSA-Earth can be extended to include offshore consumers in a standard transmission network.